In [33]:
import pandas as pd
import feedparser
from newspaper import Article
import re
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def get_contents(link):
        article = Article(link, language='ko')
        article.download()
        article.parse()
        return article.text

def cleaning_korean(content):
    content = content.replace('\n\n','')
    content = ' '.join(content.split(' '))

    #클리닝1
    content = re.sub(r'... 기자', '', content)
    content = re.sub(r'...기자', '', content)

    #괄호안 제거 
    ''' 괄호안 내용 제거하기 '''
    pattern_bracket1 = r'\([^)]*\)'
    pattern_bracket2 = r'\{[^)]*\}'
    pattern_bracket3 = r'\<[^)]*\>'
    pattern_bracket4 = r'\[[^)]*\]'

    x = '이건 {괄호 안의 불필요한 정보를} 삭제하는 코드다.'
    content = re.sub(pattern=pattern_bracket1, repl='', string= content)
    content = ' '.join(content.split())
    content = re.sub(pattern=pattern_bracket2, repl='', string= content)
    content = ' '.join(content.split())
    content = re.sub(pattern=pattern_bracket3, repl='', string= content)
    content = ' '.join(content.split())
    content = re.sub(pattern=pattern_bracket4, repl='', string= content)
    content = ' '.join(content.split())
    #클리닝
    content = content.lower() #lower case
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    content = re.sub(pattern=pattern, repl='', string=content)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    content = re.sub(pattern=pattern, repl='', string=content)
    # 특수기호 특수
    content = re.sub('[-=+#/\?:^$@*\"※~%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》Δ△·“=◎<>▷]', '', content)
    content = re.sub('[一-龥]', '',content)
    #pattern = '[^\w\s]'         # 특수기호제거
    #content = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\:\;\!\-\,\_\~\$\'\"]', '',str(content)) #remove punctuation
    content = re.sub(r"[^a-zA-Z0-9가-힣\.\s]","",content)
    content = re.sub(pattern=pattern, repl='', string=content)
    content = re.sub(r'\s+', ' ', content) #remove extra space
    content = re.sub(r'<[^>]+>','',content) #remove Html tags
    content = re.sub(r'\s+', ' ', content) #remove spaces
    content = re.sub(r"^\s+", '', content) #remove space from start
    content = re.sub(r'\s+$', '', content) #remove space from the end
    return content

In [5]:
dflink = pd.read_csv('./교보생명.csv')

In [7]:
p_contents = []
for url in dflink['url']:
    try:
        p_contents.append(  cleaning_korean(get_contents(url) ))
        print(url)
    except Exception as e:
        print(e)

http://www.investchosun.com/m/article.html?contid=2023020980230
https://www.etnews.com/20230208000108
Article `download()` failed with ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None)) on URL https://zdnet.co.kr/view/?no=20230208112704
https://mbnmoney.mbn.co.kr/news/view?news_no=MM1004854777
http://www.insweek.co.kr/59370
https://www.nocutnews.co.kr/news/5891824
https://news.einfomax.co.kr/news/articleView.html?idxno=4253692
http://news.bizwatch.co.kr/article/finance/2023/02/08/0043
https://www.straightnews.co.kr/news/articleView.html?idxno=225581
http://www.thebell.co.kr/free/Content/ArticleView.asp?key=202302091334341040105951&svccode=
https://www.mk.co.kr/news/economy/10636493
https://www.nbntv.kr/news/articleView.html?idxno=67182
https://www.biztribune.co.kr/news/articleView.html?idxno=274832
https://www.sisaon.co.kr/news/articleView.html?idxno=147360
https://www.lcnews.co.kr/news/articleView.html?idxno=44575
https://www.hankyun

In [8]:
from konlpy.tag import Mecab


def content2noun_mecab(series_content, save_path = None):
    try:
        tokenizer = Mecab() 
    except:
        tokenizer = Mecab('C:\mecab\mecab-ko-dic') 
    res_list = []
    print('------ start mecab tokenizer ------')
    for row in series_content:
        try:
            res = ' '.join(tokenizer.nouns(row))
            res_list.append(res)
        except Exception as e:
            print(e)
            res_list.append('')
    return res_list

In [9]:
pc = []
for c in p_contents:
    pc.append(content2noun_mecab(c))

------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------


In [10]:
pres = content2noun_mecab(p_contents)


pl = []
for res in pres:
    pl = pl+res.split(' ')


pclean = []
for cc in pl:
    if len(cc) < 2:
        continue
    else:
        pclean.append(cc)
        

------ start mecab tokenizer ------


---

In [11]:
from wordcloud import WordCloud

In [12]:
def get_count(pclean):
    dfp = pd.DataFrame()
    dfp['count'] = pclean
    
    dfpc = dfp.value_counts().to_frame()
    dfpc['word'] =  dfpc.index
    dfpc = dfpc.reset_index(drop=True)
    dfpc['word'] = dfpc['word'].apply(lambda x : x[0])
    dfpc.columns = ['count','word']
    
    dfpc = dfpc.loc[dfpc['count'] > 1]

    return dfpc

In [14]:
dfpc = get_count(pclean)
dfpc.to_csv('ooo.csv')

---

In [47]:
dfpc = pd.read_csv('./10개미만제거/ooo.csv', index_col = 0)
mx = dfpc['count'].max()
mn = dfpc['count'].min()
#dfpc['count'] = dfpc['count'].apply(lambda x : int( np.log2( (mx - x)/(mx - mn) + 0.0001 ) + 10 ))

dfpc['word'] = dfpc['word'].apply(lambda x : x.lstrip().rstrip())
Counter(list(dfpc['word'].values))
if len(dfpc) !=  len(set(dfpc['word'].values)):
    print('dif 12312313414124')
pdict = dict(zip(dfpc['word'] , dfpc['count']))
wc = WordCloud(font_path='malgun', width=800, height=800, scale=1.0, max_font_size=800, background_color = 'white')
gen = wc.generate_from_frequencies(pdict)
wc.to_file('oo.png')
#plt.figure()
#plt.imshow(gen)
#Counter(list(dfpc['word'].values))

In [45]:
dfpc = pd.read_csv('./10개미만제거/ooo.csv', index_col = 0)
mx = dfpc['count'].max()
mn = dfpc['count'].min()
#dfpc['count'] = dfpc['count'].apply(lambda x : int( np.log2( (mx - x)/(mx - mn) + 0.0001 ) + 10 ))

dfpc['word'] = dfpc['word'].apply(lambda x : x.lstrip().rstrip())
Counter(list(dfpc['word'].values))

Counter({'지주사': 1,
         '금융': 1,
         '전환': 1,
         '설립': 1,
         '추진': 1,
         '대주주': 1,
         '성장': 1,
         '자회사': 1,
         '사업': 1,
         '주식': 1,
         '지주회사': 1,
         '소주주': 1,
         '지분율': 1,
         '지주': 1,
         '보험': 1,
         '분할': 1,
         '불확실성': 1,
         '이사회': 1,
         '신주': 1,
         '포트폴리오': 1,
         '동력': 1,
         '생명': 1,
         '인적분할': 1,
         '교보': 1,
         '뷰카': 1,
         '어피너티': 1,
         '분쟁': 1,
         '장기': 1,
         '관계자': 1,
         '계획': 1,
         '내년': 1,
         '보험업': 1,
         '가능': 1,
         '옵션': 1,
         '투자': 1,
         '지배': 1,
         '마련': 1,
         '신창재': 1,
         '경영': 1,
         '구축': 1,
         '신설': 1,
         '과정': 1,
         '자산': 1,
         '단계': 1,
         '그룹': 1,
         '기대': 1,
         '기업': 1,
         '하반기': 1,
         '인수': 1,
         '때문': 1,
         '출범': 1,
         '전략': 1,
         '관계사': 1,
         '상장': 1,
      